# Documentación del Código para Clasificación de Frutas

Este documento describe un codigo diseñado para entrenar un modelo de clasificación de imágenes de frutas utilizando PyTorch. El proceso abarca desde la preparación de los datos hasta el entrenamiento del modelo y la visualización de su rendimiento.

---
### Paso 1: Importación de Librerías

Se importan las librerías esenciales para el entrenamiento del modelo de clasificación de frutas.

* **PyTorch (`torch`, `torch.nn`, `torch.optim`) y `torchvision`**: Fundamentales para definir el modelo de red neuronal, cargar y transformar los datos de imagen, y gestionar el proceso de entrenamiento.
* **`matplotlib.pyplot`**: Se usa para generar gráficos, específicamente la curva de pérdida, que permite visualizar el progreso del entrenamiento.
* **`os`**: Proporciona funcionalidades para interactuar con el sistema operativo, como la gestión de rutas de archivos.
* **`google.colab.drive`**: Permite montar Google Drive en el entorno de Colab para acceder a los datos almacenados.

In [ ]:
# Paso 1: Importar librerías necesarias
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os

from google.colab import drive
drive.mount('/content/drive')

---
### Paso 2: Configuración Inicial y Transformaciones

En este paso, se establecen los parámetros cruciales y se definen las transformaciones necesarias para procesar las imágenes del *dataset*.

* **`batch_size`**: Define el número de imágenes procesadas en cada iteración de entrenamiento.
* **`img_size`**: Especifica las dimensiones a las que se redimensionarán todas las imágenes.
* **`device`**: Determina si se utilizará una GPU (`cuda`) o la CPU para el entrenamiento, priorizando la GPU si está disponible.
* **`data_dir`**: La ruta donde se encuentran las imágenes del *dataset* de frutas.

Las **transformaciones** se aplican a las imágenes para asegurar uniformidad y normalización:

* **`transforms.Resize((img_size, img_size))`**: Redimensiona todas las imágenes al tamaño definido (`224x224`).
* **`transforms.ToTensor()`**: Convierte las imágenes a tensores de PyTorch.
* **`transforms.Normalize(...)`**: Normaliza los valores de los píxeles utilizando la media y desviación estándar predefinidas de ImageNet, lo que ayuda al modelo a aprender de manera más eficiente.

Luego, se carga el *dataset* completo de imágenes y se divide en conjuntos de entrenamiento (80%) y validación (20%) para evaluar el rendimiento del modelo en datos no vistos. La semilla (`torch.manual_seed(42)`) se fija para asegurar la reproducibilidad de la división. Finalmente, se crean `DataLoader` para cargar los datos en lotes durante el entrenamiento y la validación.

In [ ]:
# Paso 2: Configuración Inicial

batch_size = 8
img_size = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/content/drive/MyDrive/FruitScan/data/fruits"

# Transformaciones
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

# Cargar dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes

# Dividir dataset
val_split = 0.2
val_size = int(len(full_dataset) * val_split)
train_size = len(full_dataset) - val_size
torch.manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

---
### Paso 3: Definición del Modelo

Este paso se enfoca en la construcción y preparación del modelo de red neuronal para la tarea de clasificación.

Se utiliza un modelo **VGG19 preentrenado** en ImageNet. El uso de un modelo preentrenado permite aprovechar las características visuales generales que ya ha aprendido de un *dataset* grande y diverso, lo que es una técnica conocida como *transfer learning*.

Las **capas convolucionales** del VGG19 se **congelan** (`param.requires_grad = False`). Esto significa que sus pesos no se actualizarán durante el entrenamiento, manteniendo las características de alto nivel aprendidas.

Finalmente, se modifica la **capa final (clasificador)** del modelo. La capa original de salida de VGG19 se reemplaza por una nueva capa lineal (`nn.Linear`) con el número de salidas igual a la cantidad de clases de frutas en nuestro *dataset*. Solo los parámetros de esta nueva capa se entrenarán, adaptando el modelo a nuestra tarea específica. El modelo se mueve al dispositivo (`cuda` o `cpu`) configurado previamente.

In [ ]:
# Paso 3: Definición del Modelo

model = models.vgg19(pretrained=True)

# Congelar capas convolucionales (transfer learning)
for param in model.features.parameters():
    param.requires_grad = False

# Modificar la capa final (classifier)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(class_names))

model = model.to(device)

---
### Paso 4: Configuración de Entrenamiento

Aquí se definen los componentes clave para el proceso de entrenamiento del modelo.

* **Función de pérdida (`criterion`)**: Se elige `nn.CrossEntropyLoss()`, que es la función de pérdida estándar y más adecuada para problemas de clasificación multiclase. Mide la diferencia entre las probabilidades predichas por el modelo y las etiquetas verdaderas.
* **Optimizador (`optimizer`)**: Se utiliza `optim.Adam`. Este es un optimizador eficiente y adaptable que ajusta los pesos del modelo durante el entrenamiento. Es importante destacar que el optimizador solo actuará sobre los parámetros de la capa final (`model.classifier.parameters()`) ya que las capas convolucionales han sido congeladas. La tasa de aprendizaje (`lr=0.001`) controla el tamaño de los pasos que el optimizador da para ajustar los pesos.
* **Épocas (`epochs`)**: Se define el número de veces que el modelo recorrerá todo el *dataset* de entrenamiento. En este caso, el modelo será entrenado por 10 épocas.

In [ ]:
# Paso 4: Configuración de Entrenamiento

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
epochs = 10

---
### Paso 5: Entrenamiento del Modelo

Este es el núcleo del proceso, donde el modelo aprende de los datos.

Se inicializan dos listas, `train_losses` y `val_losses`, para almacenar el valor de la pérdida en cada época, tanto para el conjunto de entrenamiento como para el de validación, respectivamente.

El entrenamiento se itera a través del número definido de **épocas**. En cada época:

1.  **Modo de entrenamiento (`model.train()`)**: El modelo se establece en modo de entrenamiento, lo que habilita funcionalidades como *dropout* y normalización por lotes.
2.  **Bucle de entrenamiento**: Se itera sobre los lotes de imágenes y etiquetas del `train_loader`.
    * Las imágenes y etiquetas se mueven al dispositivo (`device`).
    * Los gradientes se ponen a cero (`optimizer.zero_grad()`) para evitar la acumulación de gradientes de iteraciones anteriores.
    * Se realizan predicciones (`outputs = model(images)`).
    * Se calcula la **pérdida de entrenamiento** (`loss = criterion(outputs, labels)`).
    * Se calcula el gradiente de la pérdida con respecto a los pesos del modelo (`loss.backward()`).
    * Los pesos del modelo se actualizan (`optimizer.step()`).
    * La pérdida acumulada se registra.
3.  La **pérdida promedio de la época de entrenamiento** se calcula y se almacena en `train_losses`.
4.  **Validación**:
    * El modelo se establece en modo de evaluación (`model.eval()`), deshabilitando *dropout* y normalización por lotes para una evaluación consistente.
    * Se deshabilitan los cálculos de gradientes (`with torch.no_grad():`) para ahorrar memoria y computación durante la inferencia.
    * Se itera sobre los lotes del `val_loader`, calculando la **pérdida de validación** y el número de predicciones correctas para determinar la **precisión**.
    * La **pérdida promedio de la época de validación** se calcula y se almacena en `val_losses`.
5.  Se imprime el progreso de la época, incluyendo la pérdida de entrenamiento y validación, y la precisión en el conjunto de validación.

In [ ]:
# Paso 5: Entrenamiento del Modelo

train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)

    # Validación
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    accuracy = 100 * correct / total

    print(f"\u2705 Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f} - Accuracy: {accuracy:.2f}%")

---
### Paso 6: Visualización de la Curva de Pérdida

La visualización de la curva de pérdida es una herramienta crucial para entender el comportamiento del modelo durante el entrenamiento.

Se utiliza `matplotlib.pyplot` para graficar las pérdidas de entrenamiento y validación registradas a lo largo de las épocas.

* El eje X representa las **épocas**.
* El eje Y representa el valor de la **pérdida**.
* Se trazan dos líneas: una para la **pérdida de entrenamiento** (`train_losses`) y otra para la **pérdida de validación** (`val_losses`).
* La gráfica incluye etiquetas para los ejes, un título descriptivo y una leyenda para distinguir ambas curvas.

Esta visualización ayuda a:
* **Identificar si el modelo está aprendiendo**: Si ambas curvas disminuyen, el modelo está aprendiendo.
* **Detectar sobreajuste (overfitting)**: Si la pérdida de entrenamiento sigue disminuyendo mientras la pérdida de validación comienza a aumentar, indica que el modelo está memorizando el *dataset* de entrenamiento y no generaliza bien a datos nuevos.
* **Detectar subajuste (underfitting)**: Si ambas pérdidas permanecen altas, el modelo podría no ser lo suficientemente complejo o no ha entrenado lo suficiente.

In [ ]:
# Paso 6: Visualización de la Curva de Pérdida

plt.plot(train_losses, label='Entrenamiento')
plt.plot(val_losses, label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de Pérdida')
plt.legend()
plt.show()

---
### Paso 7: Guardado del Modelo

Finalmente, se guarda el estado entrenado del modelo para su uso futuro, como la inferencia en nuevas imágenes.

El `state_dict()` del modelo (que contiene todos los parámetros aprendidos) se guarda en un archivo con el nombre `fruitscan_model.pth`. Esto permite cargar el modelo en el futuro sin necesidad de reentrenarlo.

In [ ]:
# Paso 7: Guardado del Modelo

model_path = "fruitscan_model.pth"
torch.save(model.state_dict(), model_path)
print(f"\n✅ Modelo guardado como: {model_path}")